In [5]:
#Run routeLinkedList.ipynb
%run routeLinkedList.ipynb

In [6]:
REQUESTSTATES = {
    "WAITING":0,
    "SCHEDULED":1,
    "ONGOING":2,
    "PREDICTED":3,
    "COMPLETED":4,
    "REJECTED":5
    }

VEHICLESTATES = {
    "IDLE":0,
    "MOVING":1,
    "MOVINGFULL":2
}

In [7]:
class Request:
    def __init__(self, passengerAmount, origin, destination, time):
        self.request_id = id(self)
        self.passengerAmount = passengerAmount
        self.origin = origin
        self.destination = destination
        self.timeOfRequest = time

    def __str__(self):
        return "Request: id: {}, passengerAmount: {}, origin: {}, destination: {}, time: {}".format(self.request_id, self.passengerAmount, self.origin, self.destination, self.time)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.request_id == other.request_id
    
    #Change state of request
    def changeState(self, state):
        self.state = REQUESTSTATES[state]

    #Getters
    def getId(self):
        return self.request_id
    
    def getPassengerAmount(self):
        return self.passengerAmount
    
    def getOrigin(self):
        return self.origin
    
    def getDestination(self):
        return self.destination
    
    def getTime(self):
        return self.timeOfRequest
    
    def getState(self):
        return self.state

In [8]:
class Routes:
    def __init__(self):
        self.routeList = RouteLinkedList()

    def addRequestToRoute(self, request, indexOrigin, indexDestination):
        self.routeList.insertAtIndex(request.origin, request.id, indexOrigin)
        self.routeList.insertAtIndex(request.destination, request.id, indexDestination)

    def arrivedAtDestionation(self):
        self.routeList.deleteAtStart()        

In [9]:
class Vehicle:
    def __init__(self, vehicle_id, capacity, origin):
        self.vehicle_id = vehicle_id
        self.capacity = capacity
        self.currentAmountOfPassengers = 0
        self.currentPosition = origin
        self.state = VEHICLESTATES["IDLE"]
        self.route = Routes()

    def __str__(self):
        return "Vehicle: id: {}, capacity: {}, currentPosition: {}, route: {}".format(self.vehicle_id, self.capacity, self.currentPosition, self.route.getSize())
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.vehicle_id == other.vehicle_id
    
    def addRequestToRoute(self, request, indexOrigin, indexDestination):
    
        #Check Passengers Limit
        if request.getPassengerAmount() + self.currentAmountOfPassengers > self.capacity:
            print("Request exceeds capacity of vehicle")
        else:
            #If vehicle has space add request to the route at the specified indexes
            self.route.addRequestToRoute(request, indexOrigin, indexDestination)
            self.currentAmountOfPassengers += request.getPassengerAmount()
    
    def arrivedAtNextStop(self):
        self.route.deleteNode()

In [10]:
Vehicle1 = Vehicle(1, 4, 1)